In [69]:
%timeit
import numpy as np
from scipy.spatial.distance import cdist

# Obstacle Heuristic Calculation Optimization

In [105]:
### Obstacles generation
min_x = -10
max_x = 10
min_y = -10
max_y = 10
num_obstacles = 100

obs_x = np.random.uniform(low=min_x, high=max_x, size=(num_obstacles,))
obs_y = np.random.uniform(low=min_y, high=max_y, size=(num_obstacles,))
obstacles = np.array([obs_x, obs_y]).T

# Obstacle dist tolerance
obstacle_dist_tol = 4

### Trajectory generation
N_TIMESTEPS = 60
trajectory = np.random.rand(N_TIMESTEPS, 5)
trajectory[:,0] = np.linspace(0, 20, N_TIMESTEPS)
trajectory[:,1] = np.linspace(0, 20, N_TIMESTEPS)

### Robot Stuff
robot_state = np.zeros(5)
robot_radius = 0.2

In [106]:
trajectory[0:5, 0:3]

array([[0.        , 0.        , 0.12246629],
       [0.33898305, 0.33898305, 0.0981544 ],
       [0.6779661 , 0.6779661 , 0.91926324],
       [1.01694915, 1.01694915, 0.97375598],
       [1.3559322 , 1.3559322 , 0.37657069]])

### Nearest Obstacle Calculation

In [107]:
%%time
robot_pos = robot_state[0:2]
nearest_obstacles = []
for obs in obstacles:
    # Distance from robot pose to obstacle
    euclidean_dist_to_obs = np.linalg.norm(robot_pos - obs)

    # If robot is within range of the obstacle
    if euclidean_dist_to_obs < obstacle_dist_tol:
        nearest_obstacles.append(obs)
nearest_obstacles = np.array(nearest_obstacles)
        
print("Nearest obstacles: \n", nearest_obstacles, "\n")

Nearest obstacles: 
 [[ 0.10686252  3.43440846]
 [-3.36768581 -0.32317175]
 [-3.01474173 -1.24460746]
 [-1.60443884 -0.0580862 ]
 [-0.0207941   0.26836848]
 [ 2.58059446  2.43252326]
 [-0.3001136  -2.3767408 ]
 [-3.46076292  1.10832927]
 [ 0.33814874 -0.91286807]
 [ 0.45167805 -3.75836369]
 [ 0.0783216  -3.50807287]
 [-3.44029585 -0.09640552]
 [-0.23741444  3.27302232]
 [-1.57664554 -1.8177159 ]
 [-0.21626438  3.96258939]
 [-1.51007824 -3.65652005]
 [ 1.69117032  3.0579298 ]
 [ 2.14591839  2.69530298]] 

CPU times: user 3.81 ms, sys: 0 ns, total: 3.81 ms
Wall time: 2.86 ms


In [108]:
%%time
robot_pos = robot_state[0:2]
nearest_obstacles_ind = np.where(np.linalg.norm(robot_pos - obstacles, axis=1) < obstacle_dist_tol)

nearest_obstacles = obstacles[nearest_obstacles_ind]

print("Nearest obstacles: \n", nearest_obstacles, "\n")

Nearest obstacles: 
 [[ 0.10686252  3.43440846]
 [-3.36768581 -0.32317175]
 [-3.01474173 -1.24460746]
 [-1.60443884 -0.0580862 ]
 [-0.0207941   0.26836848]
 [ 2.58059446  2.43252326]
 [-0.3001136  -2.3767408 ]
 [-3.46076292  1.10832927]
 [ 0.33814874 -0.91286807]
 [ 0.45167805 -3.75836369]
 [ 0.0783216  -3.50807287]
 [-3.44029585 -0.09640552]
 [-0.23741444  3.27302232]
 [-1.57664554 -1.8177159 ]
 [-0.21626438  3.96258939]
 [-1.51007824 -3.65652005]
 [ 1.69117032  3.0579298 ]
 [ 2.14591839  2.69530298]] 

CPU times: user 809 µs, sys: 364 µs, total: 1.17 ms
Wall time: 1.06 ms


### Obstacle Cost Calculation

In [111]:
%%time
cost = 0.0
min_dist = np.inf   # Minimum distance to obstacle

# For every trajectory point 
for i in range(trajectory.shape[0]):
    for obs in nearest_obstacles:
        # Extract the trajectory's position
        trajectory_pos = trajectory[i, 0:2]

        # Calculate the distance between the obstacle and the trajectory's position
        euclidean_dist_to_obs = np.linalg.norm(trajectory_pos - obs)

        # Collision with obstacle? 
        # Note if dist between robot position and obstacle point is less than the radius of the robot
        # Then the obstacle point is within the robot (i.e collision)


        if euclidean_dist_to_obs <= robot_radius:
            cost = np.inf
            break
            
        if euclidean_dist_to_obs < min_dist:
            min_dist = euclidean_dist_to_obs
            
if cost != np.inf:
    cost = 1.0 / min_dist
            
print("min_dist: ", min_dist)
print("obs_cost: ", cost, "\n")

min_dist:  0.21610619802894418
obs_cost:  4.627354555865469 

CPU times: user 10.3 ms, sys: 5.05 ms, total: 15.4 ms
Wall time: 11.6 ms


In [112]:
%%time
cost = 0.0
min_dist = np.inf   # Minimum distance to obstacle

trajectory_positions = trajectory[:, 0:2]
obs_x = nearest_obstacles[:,0]
obs_y = nearest_obstacles[:,1]


dx = trajectory_positions[:, 0] - obs_x[:, None]
dy = trajectory_positions[:, 1] - obs_y[:, None]
euclidean_dist = np.hypot(dx, dy)

if np.array(euclidean_dist <= robot_radius).any():
    cost = np.inf
else:
    min_dist = np.min(euclidean_dist)
    cost = 1.0 / min_dist

# for traj in trajectory_positions:
#     if (np.linalg.norm(traj - nearest_obstacles, axis=1) <= robot_radius).any():
#         cost = np.inf
#         break
    
    
print("min_dist: ", min_dist)
print("obs_cost: ", cost, "\n")

min_dist:  0.21610619802894418
obs_cost:  4.627354555865469 

CPU times: user 785 µs, sys: 0 ns, total: 785 µs
Wall time: 566 µs
